In [ ]:
# ANN召回
# 为测试集用户召回
# 通过annoy tree为所有的item构建索引，并通过测试集中所有的user embedding为每个用户召回一定数量的item
def get_DSSM_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk):
    """近邻检索，这里用annoy tree"""
    # 把doc_embs构建成索引树
    f = user_embs.shape[1]
    t = AnnoyIndex(f, 'angular')
    for i, v in enumerate(doc_embs):
        t.add_item(i, v)
    t.build(10)

    # 每个用户向量， 返回最近的TopK个item
    user_recall_items_dict = collections.defaultdict(dict)
    for i, u in enumerate(user_embs):
        recall_doc_scores = t.get_nns_by_vector(u, topk, include_distances=True)
        # recall_doc_scores是(([doc_idx], [scores]))， 这里需要转成原始doc的id
        raw_doc_scores = list(recall_doc_scores)
        raw_doc_scores[0] = [doc_idx_2_rawid[i] for i in raw_doc_scores[0]]
        # 转换成实际用户id
        user_recall_items_dict[user_idx_2_rawid[i]] = dict(zip(*raw_doc_scores))

    # 默认是分数从小到大排的序， 这里要从大到小
    user_recall_items_dict = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in user_recall_items_dict.items()}

    pickle.dump(user_recall_items_dict, open(file_path + 'DSSM_u2i_dict.pkl', 'wb'))

    return user_recall_items_dict


In [ ]:
# 测试召回结果
# 为测试集用户的召回结果进行测试
user_recall_items_dict = get_DSSM_recall_res(user_embs, item_embs, user_idx_2_rawid, doc_idx_2_rawid, topk=TOP_NUM)

test_true_items = {line[0]:line[1] for line in test_set}

s = []
precision = []
for i, uid in tqdm(enumerate(list(user_recall_items_dict.keys()))):
    # try:
    pred = [x for x, _ in user_recall_items_dict[uid]]
    filter_item = None
    recall_score = recall_N(test_true_items[uid], pred, N=TOP_NUM)
    s.append(recall_score)
    precision_score = precision_N(test_true_items[uid], pred, N=TOP_NUM)
    precision.append(precision_score)
    print("recall", np.mean(s))
    print("precision", np.mean(precision))
